In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import cosine_similarity
import os
import datetime

dados = pd.read_excel("gym recommendation.xlsx")
dados.drop(columns=['Diet', "ID"], inplace=True)
dados

,Sex,Age,Height,Weight,Hypertension,Diabetes,BMI,Level,Fitness Goal,Fitness Type,Exercises,Equipment,Recommendation
0,Male,18,1.68,47.5,No,No,16.83,Underweight,Weight Gain,Muscular Fitness,"Squats, deadlifts, bench presses, and overhead...",Dumbbells and barbells,Follow a regular exercise schedule. Adhere to ...
1,Male,18,1.68,47.5,Yes,No,16.83,Underweight,Weight Gain,Muscular Fitness,"Squats, deadlifts, bench presses, and overhead...","Light athletic shoes, resistance bands, and li...",Follow a regular exercise schedule. Adhere to ...
2,Male,18,1.68,47.5,No,Yes,16.83,Underweight,Weight Gain,Muscular Fitness,"Squats, yoga, deadlifts, bench presses, and ov...","Dumbbells, barbells and Blood glucose monitor",Follow a regular exercise schedule. Adhere to ...
3,Male,18,1.68,47.5,Yes,Yes,16.83,Underweight,Weight Gain,Muscular Fitness,"Squats, yoga, deadlifts, bench presses, and ov...","Light athletic shoes, resistance bands, light ...",Follow a regular exercise schedule. Adhere to ...
4,Male,18,1.68,47.5,No,No,16.83,Underweight,Weight Gain,Muscular Fitness,"Squats, deadlifts, bench presses, and overhead...",Dumbbells and barbells,Follow a regular exercise schedule. Adhere to ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14584,Male,35,1.88,98.0,No,No,27.73,Overweight,Weight Loss,Cardio Fitness,"Brisk walking, cycling, swimming, running , or...","Ellipticals, Indoor Rowers,Treadmills, and Row...",Follow a regular exercise schedule. Adhere to ...
14585,Male,36,1.89,99.0,No,No,27.71,Overweight,Weight Loss,Cardio Fitness,"Brisk walking, cycling, swimming, running , or...","Ellipticals, Indoor Rowers,Treadmills, and Row...",Follow a regular exercise schedule. Adhere to ...
14586,Male,37,1.90,100.0,No,No,27.70,Overweight,Weight Loss,Cardio Fitness,"Brisk walking, cycling, swimming, running , or...","Ellipticals, Indoor Rowers,Treadmills, and Row...",Follow a regular exercise schedule. Adhere to ...
14587,Male,38,1.91,101.0,No,No,27.69,Overweight,Weight Loss,Cardio Fitness,"Brisk walking, cycling, swimming, running , or...","Ellipticals, Indoor Rowers,Treadmills, and Row...",Follow a regular exercise schedule. Adhere to ...


In [7]:


dados.columns = ['Sexo', 'Idade', 'Altura', 'Peso', 'Hipertensao', 'Diabetes', 'IMC', 'Nivel', 'Objetivo', 'Tipo_Fitness', 'Exercicios', 'Dieta', 'Equipamento']

# Label Encoding nas colunas categóricas
label_enc = LabelEncoder()
for col in ['Sexo', 'Hipertensao', 'Diabetes', 'Nivel', 'Objetivo', 'Tipo_Fitness']:
    dados[col] = label_enc.fit_transform(dados[col])

# Normalização das colunas numéricas
scaler = StandardScaler()
dados[['Idade', 'Altura', 'Peso', 'IMC']] = scaler.fit_transform(dados[['Idade', 'Altura', 'Peso', 'IMC']])

# Features e targets
colunas_features = ['Sexo', 'Idade', 'Altura', 'Peso', 'Hipertensao', 'Diabetes', 'IMC', 'Nivel', 'Objetivo', 'Tipo_Fitness']
X = dados[colunas_features]
y_exercicio = dados['Exercicios']
y_dieta = dados['Dieta']
y_equip = dados['Equipamento']

# 2. Função de recomendação híbrida
def recomendador_hibrido(entrada_usuario, historico_usuario=None):
    entrada_df = pd.DataFrame([entrada_usuario])
    entrada_df[['Idade', 'Altura', 'Peso', 'IMC']] = scaler.transform(entrada_df[['Idade', 'Altura', 'Peso', 'IMC']])
    entrada_features = entrada_df[colunas_features]

    if historico_usuario is not None and len(historico_usuario) >= 5:
        hist_df = pd.DataFrame(historico_usuario)
        hist_df[['Idade', 'Altura', 'Peso', 'IMC']] = scaler.transform(hist_df[['Idade', 'Altura', 'Peso', 'IMC']])
        hist_features = hist_df[colunas_features]

        # Similaridade média com histórico
        sim = cosine_similarity(hist_features, entrada_features).mean()
        entrada_features['similaridade_cosseno'] = sim

        X_ext = X.copy()
        X_ext['similaridade_cosseno'] = cosine_similarity(X[colunas_features], entrada_features[colunas_features]).flatten()

        knn_ex = KNeighborsClassifier(n_neighbors=3).fit(X_ext, y_exercicio)
        knn_dieta = KNeighborsClassifier(n_neighbors=3).fit(X_ext, y_dieta)
        knn_equip = KNeighborsClassifier(n_neighbors=3).fit(X_ext, y_equip)

        rec_ex = knn_ex.predict(entrada_features)[0]
        rec_dieta = knn_dieta.predict(entrada_features)[0]
        rec_equip = knn_equip.predict(entrada_features)[0]

    else:
        # Apenas cosine similarity
        scores_sim = cosine_similarity(X, entrada_features).flatten()
        idx_top = scores_sim.argsort()[-5:][::-1]
        dados_similares = dados.iloc[idx_top]

        rec_ex = dados_similares['Exercicios'].mode()[0]
        rec_dieta = dados_similares['Dieta'].mode()[0]
        rec_equip = dados_similares['Equipamento'].mode()[0]

    recomendacao = {
        'Data_Hora': datetime.datetime.now().isoformat(),
        'Sexo': entrada_usuario['Sexo'],
        'Idade': entrada_usuario['Idade'],
        'Altura': entrada_usuario['Altura'],
        'Peso': entrada_usuario['Peso'],
        'Hipertensao': entrada_usuario['Hipertensao'],
        'Diabetes': entrada_usuario['Diabetes'],
        'IMC': entrada_usuario['IMC'],
        'Nivel': entrada_usuario['Nivel'],
        'Objetivo': entrada_usuario['Objetivo'],
        'Tipo_Fitness': entrada_usuario['Tipo_Fitness'],
        'Recomendacao_Exercicio': rec_ex,
        'Recomendacao_Dieta': rec_dieta,
        'Recomendacao_Equipamento': rec_equip
    }


    caminho_csv = "registro_recomendacoes.csv"
    df_saida = pd.DataFrame([recomendacao])
    if os.path.exists(caminho_csv):
        df_saida.to_csv(caminho_csv, mode='a', header=False, index=False)
    else:
        df_saida.to_csv(caminho_csv, index=False)

    return recomendacao

# 3. Exemplo de uso:
novo_usuario = {
    'Sexo': 1,
    'Idade': 28,
    'Altura': 1.75,
    'Peso': 70,
    'Hipertensao': 0,
    'Diabetes': 0,
    'IMC': 22.9,
    'Nivel': 0,
    'Objetivo': 1,
    'Tipo_Fitness': 0
}

# Sem histórico por enquanto
historico_usuario = None

# Obter recomendação
recomendacao = recomendador_hibrido(novo_usuario, historico_usuario)
print("Recomendação gerada:")
print(recomendacao)


Recomendação gerada:
{'Data_Hora': '2025-06-24T23:02:23.405913', 'Sexo': 1, 'Idade': 28, 'Altura': 1.75, 'Peso': 70, 'Hipertensao': 0, 'Diabetes': 0, 'IMC': 22.9, 'Nivel': 0, 'Objetivo': 1, 'Tipo_Fitness': 0, 'Recomendacao_Exercicio': 'Squats, deadlifts, bench presses, and overhead presses', 'Recomendacao_Dieta': 'Dumbbells and barbells', 'Recomendacao_Equipamento': "Follow a regular exercise schedule. Adhere to the exercise and diet plan to get better results. It is important to approach weight gain in a healthy and balanced way, focusing on both exercise and nutrition. Keep in mind that weight gain should be gradual and focused on building lean muscle rather than increasing fat. Additionally, it's always a good idea to consult with a health care professional or registered dietitian before making any significant changes to your exercise or diet plan. Here are some important tips:- Stay hydrated by drinking enough water throughout the day. Monitor your progress and adjust your diet and